In [96]:
import json
import os
import pandas as pd
from datetime import datetime
import pytz
import itertools

In [97]:
GAME = "hackaburg-campuswiese"

In [98]:
df = pd.read_parquet(os.path.join("../data", GAME, "log-interpol/interpol.parquet"))

In [99]:
df.head()

team_name     3d759ca3-19d7-11ef-9bd3-e50b875d9764             \
dim                                            lat        lon   
1716560477645                            49.002177  12.099576   
1716560478645                            49.002177  12.099565   
1716560479645                            49.002177  12.099553   
1716560480645                            49.002177  12.099542   
1716560481645                            49.002177  12.099530   

team_name     212797ed-19d7-11ef-9397-591084319220             \
dim                                            lat        lon   
1716560477645                            49.002072  12.099348   
1716560478645                            49.002073  12.099340   
1716560479645                            49.002074  12.099334   
1716560480645                            49.002075  12.099328   
1716560481645                            49.002076  12.099323   

team_name     5fa28584-19d7-11ef-89a9-91251b4272d3             \
dim                                            lat        lon   
1716560477645                            49.002101  12.099363   
1716560478645                            49.002100  12.099360   
1716560479645                            49.002099  12.099358   
1716560480645                            49.002098  12.099356   
1716560481645                            49.002098  12.099354   

team_name     2a539389-19d7-11ef-afee-e50b875d9764             
dim                                            lat        lon  
1716560477645                            49.002248  12.099911  
1716560478645                            49.002248  12.099900  
1716560479645                            49.002248  12.099890  
1716560480645                            49.002248  12.099879  
1716560481645                            49.002248  12.099868

In [100]:
# get list of teams
teams = list(set(c[0] for c in df.columns))
teams

['3d759ca3-19d7-11ef-9bd3-e50b875d9764',
 '5fa28584-19d7-11ef-89a9-91251b4272d3',
 '2a539389-19d7-11ef-afee-e50b875d9764',
 '212797ed-19d7-11ef-9397-591084319220']

In [101]:
# get catch data
data_caught = json.load(open(os.path.join('../data/', GAME, "log-export/team_caught.json")))

In [102]:
# dictionary of caught timestamps
caught_timestamps = {}
for entry in data_caught:
    entry['timestamp'] = entry['timestamp'][:-4]
    entry['timestamp'] = entry['timestamp']
    dt = datetime.strptime(entry['timestamp'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    caught_timestamps[entry['runaway_active_user']] = int(dt.timestamp() * 1000)
# initial hunting team
entry = data_caught[-1]
caught_timestamps

{'2a539389-19d7-11ef-afee-e50b875d9764': 1716563030000,
 '3d759ca3-19d7-11ef-9bd3-e50b875d9764': 1716562033000}

In [103]:
# get initial hunter
initial_hunter = entry['hunter_active_user']
caught_timestamps[initial_hunter] = 0

In [104]:
# find intervals where hunters are close to prey
interesting_timestamps = []
interesting_start = 0
interesting = False
for t in df.index:
    hunters = [initial_hunter] + [team for team in teams if team in caught_timestamps.keys() and caught_timestamps[team] < t]
    preys = [team for team in teams if team not in hunters]
    
    # calculate pairwise distances between hunters and prey
    still_interesting = False
    for hunter, prey in itertools.product(hunters, preys):
        d_lat = df[hunter, 'lat'][t] - df[prey, 'lat'][t]
        d_lon = df[hunter, 'lon'][t] - df[prey, 'lon'][t]
        d = ((d_lat*1000)**2 + (d_lon*1000)**2)**0.5
        if d < 0.5:
            if not interesting:
                interesting_start = t
            still_interesting = True
            interesting = True
            break
    
    # end of interesting period
    if interesting and not still_interesting:
        interesting_timestamps.append((interesting_start, t))
        interesting = False
        
interesting_timestamps.append((interesting_start, t))

In [105]:
interesting_timestamps

[(1716560477645, 1716561133645),
 (1716561971645, 1716561993645),
 (1716562015645, 1716562033645),
 (1716562995645, 1716563030645),
 (1716562995645, 1716563175645)]

In [106]:
# load game running interval
running_interval = json.load(open(os.path.join('../data', GAME, "running_interval.json")))
running_interval

{'start': 1716561111000, 'end': 1716563174000}

In [107]:
# filter game running interval
interesting_timestamps = [ts for ts in interesting_timestamps if ts[0] > running_interval['start'] and ts[1] < running_interval['end']]
interesting_timestamps

[(1716561971645, 1716561993645),
 (1716562015645, 1716562033645),
 (1716562995645, 1716563030645)]